# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()


normalized_histos = list()

Welcome to JupyROOT 6.24/02


In [2]:
# %load python/drawingTools.py


In [3]:
# %load samples.py

samples = [
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v149E.68Al', 'ref'),
]


In [4]:
for hf in samples:
    hf.print_primitives()

<HistoFile ../plots1/histos_nugun_alleta_pu200_rate_v149E.68Al.root, ref>
--- HistoClass: 
['RateHistos']
--- TPs: 
['PFNFtkEleEE', 'tkEleEE', 'tkEmEE', 'PFNFtkEmEE']
--- TP selections:
{   'PFNFtkEleEE': [   'EGq1EtaBC',
                       'EGq1EtaBCD',
                       'EGq2or3EtaBCD',
                       'EGq1or3EtaBC',
                       'EGq2EtaBC',
                       'EGq3EtaBC',
                       'EGq6',
                       'EGq3',
                       'EGq1',
                       'EGq2or3',
                       'EGq6EtaBC',
                       'EGq1or3EtaBCD',
                       'EGq1or3',
                       'EGq6EtaBCD',
                       'EGq2EtaBCD',
                       'EGq3EtaBCD',
                       'EGq2',
                       'EGq2or3EtaBC'],
    'PFNFtkEmEE': [   'EGq6EtaBCD',
                      'EGq2EtaBCD',
                      'EGq2',
                      'EGq1EtaBCD',
                      'EGq3EtaBCD

In [5]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]






In [6]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [7]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.RateHistos)


--- RateHistos
# of plots: 144
# of primitives: 48
CPU times: user 195 ms, sys: 9.51 ms, total: 204 ms
Wall time: 205 ms


In [8]:
hplot.data['sample'].unique()

array(['ref'], dtype=object)

In [9]:
hplot.data['tp'].unique()

array(['PFNFtkEleEE', 'tkEleEE', 'tkEmEE', 'PFNFtkEmEE'], dtype=object)

In [10]:
hplot.data[hplot.data.tp == 'tkEleEE'].tp_sel.unique()

array(['EGq4', 'EGq4EtaBCD', 'EGq4EtaBC', 'EGq5EtaBC', 'EGq5',
       'EGq5EtaBCD'], dtype=object)

In [11]:
# %load python/rate_utilities.py
import ROOT


def convertRateToGraph(histo_rate, params=None):
    """
    Converts a rate TH1F to a graph.
    Optionally applyes a linear scale mapping to the X axis.
    """
    
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


def cutAtRate(histo, rate):
    """
    Determines the X value (pt cut) corresponding to Y value (rate)
    """
    cut = None
    for ix in range(1, histo.GetNbinsX()+1):
        if histo.GetBinContent(ix) <= rate:
            cut = histo.GetXaxis().GetBinLowEdge(ix)
            break
    return cut


In [16]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['tkEmEE'], 'EGq4EtaBC', None)
dm.addHistos([his.h_pt for his in hsets], labels=['simu'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE'], ['EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['emu'])

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




0.0 60.0
0.0 60.0
0.0 60.0


In [17]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['tkEmEE'], 'EGq4EtaBCD', None)
dm.addHistos([his.h_pt for his in hsets], labels=['simu'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE'], ['EGq1EtaBCD'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['emu'])

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [19]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['tkEleEE'], 'EGq4EtaBC', None)
dm.addHistos([his.h_pt for his in hsets], labels=['simu'])


hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEE'], ['EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['emu'])

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


## Online to offline scaling

In [ ]:
# NOTE: this version of the scaling params is clearly obsolete. It is here just to keep track of how to 
# apply online to offline scaling
# version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format('v142A'))

In [ ]:
scaling_params

In [ ]:
# eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

In [ ]:
tps = hplot.data.tp.unique().tolist()

for sample in samples:
    for tp in tps:
        for tp_sel in ['EGq4EtaBC']:
            hsets, labels, text = hplot.get_histo(histos.RateHistos, sample.type, 'PU200', tp, tp_sel, None)
            print(hsets)
            if hsets is None:
                continue
            for his in hsets:
                his.normalize(31000)
                his.h_pt_graph(
                    'online', 
                    'rate [kHz]', 
                    convertRateToGraph)
                his.h_pt_graph(
                    'offline', 
                    'rate [kHz]', 
                    lambda x: convertRateToGraph(
                        x, 
                        eval(scaling_params[
                                (scaling_params['sample'] == 'ele-'+sample.type) &
                                (scaling_params.pu == 'PU200') & 
                                (scaling_params.tp == tp) & 
                                (scaling_params.tp_sel == tp_sel)].params.iloc[0])))

In [ ]:
for sample_scaling, sample_rate, tp, tp_sel, tp_scaling, tp_sel_scaling in [
    ('ele-V9', 'TDR', 'EG', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'EGBRL', 'EtaF', 'EGBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleELBRL', 'EtaF', 'TkEleELBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleEL', 'EGq5EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'Iso0p1EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'EtaFIso0p1', 'TkEleELBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'EgEE', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('dele-NM', 'NM', 'EgEB', 'EtaF', 'EGBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEB', 'EtaF', 'tkEleEB', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEE', 'EGq5EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'Iso0p1EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'EtaFIso0p1', 'tkEleEB', 'allEtaF'),

]:
    
        
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_rate, 'PU200', tp, tp_sel, None)
    if hsets is None:
        continue
    for his in hsets:
        his.normalize(31000)
        
        his.h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        his.h_pt_graph(
            'offline', 
            'rate [kHz]', 
            lambda x: convertRateToGraph(
                x, 
                eval(scaling_params[
                        (scaling_params['sample'] == sample_scaling) &
                        (scaling_params.pu == 'PU200') & 
                        (scaling_params.tp == tp_scaling) & 
                        (scaling_params.tp_sel == tp_sel_scaling)].params.iloc[0])))

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11', 'TDR'], 'PU200', ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], 'EmEtaBC', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


## Compute fixed rate WPs

In [ ]:
rate_points = [20, 100, 1000]

In [ ]:
import pprint as pp

iso_pt_wps = {}

histo_primitives = hplot.data

tp_sels = [
    ('PFNFtkEmEB', histo_primitives[histo_primitives.tp == 'PFNFtkEmEB'].tp_sel.unique().tolist()),
    ('PFNFtkEmEE', [sel for sel in histo_primitives[histo_primitives.tp == 'PFNFtkEmEE'].tp_sel.unique().tolist() if 'EtaBC' not in sel]),   
]
    
# print (tp_sels)

for tp, tp_sel_list in tp_sels:
    iso_pt_wps[tp] = {}
    for sel in tp_sel_list:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, ['ref'], 'PU200', tp, sel, None)
        
        for his in hsets:
            his.normalize(31000)
        rate_cuts = {}
        for rate in rate_points:
            rate_cuts[rate] = cutAtRate(hsets[0].h_pt, rate)
        iso_pt_wps[tp][sel] = rate_cuts

pp.pprint(iso_pt_wps)

import json
with open('iso_pt_wps.json', 'w') as fp:
    json.dump(iso_pt_wps, fp)

In [ ]:
import tabulate
tp = 'PFNFtkEmEB'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


In [ ]:
import tabulate
tp = 'PFNFtkEmEE'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'EGq5{iso_var}WP{iso_wp}Pt{pt_wp}EtaABC'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['EGq5EtaABC'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')
